<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sd_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!nvidia-smi
import os
import subprocess
import sys
import shutil
import re

# Проверяем, не установлено ли уже
if not os.path.exists('/content/installed_tile_upscaler'):
    print("=" * 70)
    print("📦 INSTALLING TILE UPSCALER")
    print("=" * 70)

    # Устанавливаем diffusers и связанные пакеты
    print("\n[1/4] Installing Python packages...")
    packages = [
        'diffusers',
        'transformers',
        'accelerate',
        'safetensors',
        'invisible-watermark',
        'huggingface_hub'
    ]

    for package in packages:
        print(f"  → {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("  ✓ Python packages installed")

    # Устанавливаем Real-ESRGAN
    print("\n[2/4] Installing Real-ESRGAN...")

    if not os.path.exists('/content/Real-ESRGAN'):
        print("  → Cloning repository...")
        get_ipython().system('git clone -q https://github.com/xinntao/Real-ESRGAN.git /content/Real-ESRGAN')

    os.chdir('/content/Real-ESRGAN')

    print("  → Installing dependencies...")
    get_ipython().system('pip install -q basicsr facexlib gfpgan -r requirements.txt')
    get_ipython().system('python setup.py develop > /dev/null 2>&1')

    print("  → Fixing compatibility issues...")

    # URL новой модели GFPGAN
    new_model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'

    # Открываем файл inference_realesrgan.py и изменяем путь к модели
    inference_file = '/content/Real-ESRGAN/inference_realesrgan.py'
    if os.path.exists(inference_file):
        with open(inference_file, 'r') as f:
            script_content = f.read()

        # Изменяем путь к модели в строке скрипта
        new_script_content = re.sub(r"(model_path\s*=\s*[\"\']).*?([\"\'])", rf"\g<1>{new_model_path}\g<2>", script_content)

        with open(inference_file, 'w') as f:
            f.write(new_script_content)

    # Определяем версию Python
    python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
    degradations_path = f'/usr/local/lib/{python_version}/dist-packages/basicsr/data/degradations.py'

    # Делаем бэкап
    if os.path.exists(degradations_path):
        get_ipython().system(f"cp -n {degradations_path} {degradations_path}.backup")

    # Фиксим импорт rgb_to_grayscale
    get_ipython().system(f"sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' {degradations_path}")

    # Записываем полный код degradations
    degradations_code = '''import cv2
import math
import numpy as np
import random
import torch
from torch.nn import functional as F

def random_add_gaussian_noise_pt(img, sigma_range=(0, 1.0), gray_prob=0, noise_gray_prob=0, clip=True, rounds=False):
    noise_sigma = random.uniform(*sigma_range)

    if random.random() < gray_prob:
        img = rgb_to_grayscale(img)

    if random.random() < noise_gray_prob:
        noise = torch.randn(*img.shape[1:], device=img.device) * noise_sigma
        noise = noise.unsqueeze(0).repeat(img.shape[0], 1, 1)
    else:
        noise = torch.randn_like(img) * noise_sigma

    out = img + noise

    if clip and rounds:
        out = torch.clamp((out * 255.0).round(), 0, 255) / 255.
    elif clip:
        out = torch.clamp(out, 0, 1)
    elif rounds:
        out = (out * 255.0).round() / 255.

    return out

def random_add_poisson_noise_pt(img, scale_range=(0, 1.0), gray_prob=0, clip=True, rounds=False):
    scale = random.uniform(*scale_range)

    if random.random() < gray_prob:
        img = rgb_to_grayscale(img)

    noise = torch.poisson(img * scale) / scale - img
    out = img + noise

    if clip and rounds:
        out = torch.clamp((out * 255.0).round(), 0, 255) / 255.
    elif clip:
        out = torch.clamp(out, 0, 1)
    elif rounds:
        out = (out * 255.0).round() / 255.

    return out

def rgb_to_grayscale(img):
    if img.shape[0] != 3:
        raise ValueError('Input image must have 3 channels')
    rgb_weights = torch.tensor([0.2989, 0.5870, 0.1140], device=img.device)
    grayscale = torch.sum(img * rgb_weights.view(-1, 1, 1), dim=0, keepdim=True)
    return grayscale

def circular_lowpass_kernel(cutoff, kernel_size, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size
    assert pad_to >= kernel_size, 'Pad size must be larger than kernel size'

    def _scaled_sinc(x):
        if x == 0:
            return torch.tensor(1.)
        x = x * math.pi
        return torch.sin(x) / x

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)
    dist = torch.sqrt(x**2 + y**2)

    kernel = _scaled_sinc(dist * cutoff)
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel

def random_mixed_kernels(
        kernel_list,
        kernel_prob,
        kernel_size=21,
        blur_sigma=0.1,
        blur_sigma_min=0.1,
        blur_sigma_max=10.0,
        blur_kernel_size=21,
        pad_to=0):
    num_kernels = len(kernel_list)
    kernel_type = np.random.choice(kernel_list, p=kernel_prob)

    if pad_to == 0:
        pad_to = kernel_size

    if kernel_type == 'iso':
        sigma = np.random.uniform(blur_sigma_min, blur_sigma_max)
        kernel = _generate_isotropic_gaussian_kernel(kernel_size, sigma, pad_to)
    elif kernel_type == 'aniso':
        sigma_x = np.random.uniform(blur_sigma_min, blur_sigma_max)
        sigma_y = np.random.uniform(blur_sigma_min, blur_sigma_max)
        rotation = np.random.uniform(-np.pi, np.pi)
        kernel = _generate_anisotropic_gaussian_kernel(kernel_size, sigma_x, sigma_y, rotation, pad_to)
    else:  # general
        kernel = circular_lowpass_kernel(blur_sigma, kernel_size, pad_to)

    return kernel

def _generate_isotropic_gaussian_kernel(kernel_size=21, sigma=0.1, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)
    kernel = torch.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel

def _generate_anisotropic_gaussian_kernel(kernel_size=21, sigma_x=0.1, sigma_y=0.1, rotation=0, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)

    cos_theta = torch.cos(torch.tensor(rotation))
    sin_theta = torch.sin(torch.tensor(rotation))
    x_rot = cos_theta * x - sin_theta * y
    y_rot = sin_theta * x + cos_theta * y

    kernel = torch.exp(-(x_rot**2 / (2 * sigma_x**2) + y_rot**2 / (2 * sigma_y**2)))
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel
'''

    with open(degradations_path, 'w') as f:
        f.write(degradations_code)

    # Патчим utils.py для поддержки разных форматов моделей RealESRGAN
    utils_path = '/content/Real-ESRGAN/realesrgan/utils.py'
    if os.path.exists(utils_path):
        with open(utils_path, 'r') as f:
            utils_content = f.read()

        # Ищем и заменяем проблемную часть
        old_code = '''        else:
            keyname = 'params'
        model.load_state_dict(loadnet[keyname], strict=True)'''

        new_code = '''        else:
            keyname = 'params'

        # Support different checkpoint formats
        if keyname in loadnet:
            model.load_state_dict(loadnet[keyname], strict=True)
        else:
            # Try to load directly if it's already a state_dict
            model.load_state_dict(loadnet, strict=True)'''

        if old_code in utils_content:
            utils_content = utils_content.replace(old_code, new_code)
            with open(utils_path, 'w') as f:
                f.write(utils_content)

    print("  ✓ Real-ESRGAN installed")

    # Создаем структуру папок для Tile-Upscaler
    print("\n[3/4] Creating project structure...")

    if not os.path.exists('/content/Tile-Upscaler'):
        get_ipython().system('git clone -q https://github.com/gokayfem/Tile-Upscaler.git /content/Tile-Upscaler')

    os.chdir('/content/Tile-Upscaler')

    for folder in ['models/ControlNet', 'models/models/Stable-diffusion', 'models/VAE',
                   'models/embeddings', 'models/Lora', 'models/upscalers']:
        os.makedirs(folder, exist_ok=True)
    print("  ✓ Directories created")

    # Скачиваем модели используя hf_hub_download
    print("\n[4/4] Downloading AI models from Hugging Face...")
    print("  (This will take 5-10 minutes)")

    from huggingface_hub import hf_hub_download

    models = {
        "Stable Diffusion Model": {
            "repo_id": "dantea1118/juggernaut_reborn",
            "filename": "juggernaut_reborn.safetensors",
            "local_dir": "models/models/Stable-diffusion"
        },
        "RealESRGAN x2": {
            "repo_id": "ai-forever/Real-ESRGAN",
            "filename": "RealESRGAN_x2.pth",
            "local_dir": "models/upscalers/"
        },
        "RealESRGAN x4": {
            "repo_id": "ai-forever/Real-ESRGAN",
            "filename": "RealESRGAN_x4.pth",
            "local_dir": "models/upscalers/"
        },
        "Embedding (BadPrompt)": {
            "repo_id": "philz1337x/embeddings",
            "filename": "verybadimagenegative_v1.3.pt",
            "local_dir": "models/embeddings"
        },
        "Embedding (Juggernaut Neg)": {
            "repo_id": "philz1337x/embeddings",
            "filename": "JuggernautNegative-neg.pt",
            "local_dir": "models/embeddings"
        },
        "LoRA (SDXLrender)": {
            "repo_id": "philz1337x/loras",
            "filename": "SDXLrender_v2.0.safetensors",
            "local_dir": "models/Lora"
        },
        "LoRA (More Details)": {
            "repo_id": "philz1337x/loras",
            "filename": "more_details.safetensors",
            "local_dir": "models/Lora"
        },
        "ControlNet Tile": {
            "repo_id": "lllyasviel/ControlNet-v1-1",
            "filename": "control_v11f1e_sd15_tile.pth",
            "local_dir": "models/ControlNet"
        },
        "VAE": {
            "repo_id": "stabilityai/sd-vae-ft-mse-original",
            "filename": "vae-ft-mse-840000-ema-pruned.safetensors",
            "local_dir": "models/VAE"
        }
    }

    failed_downloads = []

    for model_name, model_info in models.items():
        file_path = os.path.join(model_info["local_dir"], model_info["filename"])

        if not os.path.exists(file_path):
            print(f"\n  → Downloading {model_name}...")
            try:
                hf_hub_download(
                    repo_id=model_info["repo_id"],
                    filename=model_info["filename"],
                    local_dir=model_info["local_dir"]
                )
                if os.path.exists(file_path):
                    size_mb = os.path.getsize(file_path) / (1024*1024)
                    print(f"    ✓ Downloaded ({size_mb:.1f} MB)")
                else:
                    print(f"    ✗ Download failed")
                    failed_downloads.append(model_name)
            except Exception as e:
                print(f"    ✗ Error: {str(e)}")
                failed_downloads.append(model_name)
        else:
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"  ✓ {model_name} already exists ({size_mb:.1f} MB)")

    # Проверка результатов
    print("\n" + "=" * 70)
    print("VERIFYING INSTALLATION")
    print("=" * 70)

    all_files = [
        'models/ControlNet/control_v11f1e_sd15_tile.pth',
        'models/models/Stable-diffusion/juggernaut_reborn.safetensors',
        'models/VAE/vae-ft-mse-840000-ema-pruned.safetensors',
        'models/embeddings/verybadimagenegative_v1.3.pt',
        'models/embeddings/JuggernautNegative-neg.pt',
        'models/Lora/SDXLrender_v2.0.safetensors',
        'models/Lora/more_details.safetensors',
        'models/upscalers/RealESRGAN_x2.pth',
        'models/upscalers/RealESRGAN_x4.pth'
    ]

    missing_files = []
    total_size = 0

    print("\nInstalled files:")
    for file in all_files:
        if os.path.exists(file):
            size_mb = os.path.getsize(file) / (1024*1024)
            total_size += size_mb
            print(f"  ✓ {os.path.basename(file)} ({size_mb:.1f} MB)")
        else:
            print(f"  ✗ {os.path.basename(file)} - MISSING")
            missing_files.append(file)

    print(f"\nTotal size: {total_size:.1f} MB")

    if missing_files or failed_downloads:
        print("\n" + "=" * 70)
        print("⚠️  INSTALLATION INCOMPLETE")
        print("=" * 70)
        if failed_downloads:
            print("\nFailed downloads:")
            for item in failed_downloads:
                print(f"  • {item}")
        if missing_files:
            print("\nMissing files:")
            for file in missing_files:
                print(f"  • {os.path.basename(file)}")
        print("\n→ Please run this cell again to retry downloading missing files.")
        print("=" * 70)
    else:
        # Создаем маркер успешной установки
        open('/content/installed_tile_upscaler', 'w').close()
        print("\n" + "=" * 70)
        print("✅ INSTALLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print("\nYou can now proceed to:")
        print("  → Cell 2: Upload your image")
        print("  → Cell 3: Configure settings")
        print("  → Cell 4: Process image")
        print("  → Cell 5: Visualize results")
        print("  → Cell 6: Download result")
        print("=" * 70)
else:
    print("=" * 70)
    print("✅ ALREADY INSTALLED")
    print("=" * 70)
    os.chdir('/content/Tile-Upscaler')

    # Показываем информацию о установленных моделях
    print("\nInstalled models:")
    all_files = [
        'models/ControlNet/control_v11f1e_sd15_tile.pth',
        'models/models/Stable-diffusion/juggernaut_reborn.safetensors',
        'models/VAE/vae-ft-mse-840000-ema-pruned.safetensors',
        'models/embeddings/verybadimagenegative_v1.3.pt',
        'models/embeddings/JuggernautNegative-neg.pt',
        'models/Lora/SDXLrender_v2.0.safetensors',
        'models/Lora/more_details.safetensors',
        'models/upscalers/RealESRGAN_x2.pth',
        'models/upscalers/RealESRGAN_x4.pth'
    ]

    total_size = 0
    for file in all_files:
        if os.path.exists(file):
            size_mb = os.path.getsize(file) / (1024*1024)
            total_size += size_mb
            print(f"  ✓ {os.path.basename(file)} ({size_mb:.1f} MB)")

    print(f"\nTotal size: {total_size:.1f} MB")
    print("\n→ To reinstall: !rm /content/installed_tile_upscaler")
    print("=" * 70)

# Добавляем пути
if '/content/Real-ESRGAN' not in sys.path:
    sys.path.insert(0, '/content/Real-ESRGAN')

print("\n✓ Environment ready")

In [ ]:
#@title ##**Upload Image** { display-mode: "form" }

from google.colab import files
from PIL import Image
import io

print("📁 Please select an image to upload...")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    input_image = Image.open(io.BytesIO(uploaded[filename]))

    input_image.save('/content/input_image.png')

    print(f"✅ Image uploaded successfully: {filename}")
    print(f"   Size: {input_image.size}")

    #display(input_image)
else:
    print("❌ No image uploaded!")

In [ ]:
#@title ##**Configuration** { display-mode: "form" }

#@markdown ### Generation Settings
num_inference_steps = 50 #@param {type:"slider", min:10, max:100, step:5}
guidance_scale = 5 #@param {type:"slider", min:1, max:20, step:0.5}
scheduler = "DPM++ 3M SDE Karras" #@param ["DDIM", "DPM++ 3M SDE Karras", "DPM++ 3M Karras"]

#@markdown ### Inpainting Settings
inpaint_strength = 0.85 #@param {type:"slider", min:0.1, max:1.0, step:0.05}
inpaint_controlnet_strength = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
mask_blur = 8 #@param {type:"slider", min:0, max:50, step:2}

#@markdown ### Prompts
inpaint_prompt = "nude" #@param {type:"string"}
inpaint_negative_prompt = "" #@param {type:"string"}
custom_prompt = "Krystal Boyd nude by Luis Royo" #@param {type:"string"}
custom_negative_prompt = "" #@param {type:"string"}

#@markdown ### Effects
face_enhance = False #@param {type:"boolean"}

#@markdown ### Tiling (for large images)
use_tiles = False #@param {type:"boolean"}
tile_size = 768 #@param {type:"slider", min:512, max:1024, step:64}
tile_overlap = 128 #@param {type:"slider", min:64, max:256, step:32}

print("⚙️ Inpainting configuration saved!")
print(f"Steps: {num_inference_steps}")
print(f"Guidance Scale: {guidance_scale}")
print(f"Scheduler: {scheduler}")
print(f"Inpaint Strength: {inpaint_strength}")
print(f"ControlNet Strength: {inpaint_controlnet_strength}")
print(f"Mask Blur: {mask_blur}")
print(f"Face Enhancement: {'Enabled' if face_enhance else 'Disabled'}")
print(f"Use Tiles: {use_tiles}")
if use_tiles:
    print(f"  Tile Size: {tile_size}")
    print(f"  Tile Overlap: {tile_overlap}")
if inpaint_prompt:
    print(f"Inpaint Prompt: {inpaint_prompt}")

In [ ]:
#@title ##**Mask** { display-mode: "form" }

from google.colab import output
from IPython.display import display, HTML, Image as IPImage
from PIL import Image, ImageDraw
import base64
from io import BytesIO
import numpy as np

brush_size = 40  #@param {type:"slider", min:10, max:200, step:5}
brush_opacity = 0.5  #@param {type:"slider", min:0.1, max:1.0, step:0.1}

input_image = Image.open('/content/input_image.png')

buffered = BytesIO()
input_image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue()).decode()

html_code = f"""
<html>
<head>
<style>
    #container {{
        position: relative;
        display: inline-block;
    }}
    #imageCanvas, #maskCanvas {{
        position: absolute;
        top: 0;
        left: 0;
        cursor: crosshair;
    }}
    #maskCanvas {{
        opacity: 0.5;
    }}
    .controls {{
        margin: 10px 0;
    }}
    button {{
        padding: 10px 20px;
        margin: 5px;
        font-size: 14px;
        cursor: pointer;
    }}
</style>
</head>
<body>
<div class="controls">
    <button onclick="clearMask()">🗑️ Clear Mask</button>
    <button onclick="saveMask()">💾 Save Mask</button>
    <button onclick="toggleMaskVisibility()">👁️ Toggle Mask</button>
    <span id="status" style="margin-left: 20px; font-weight: bold;"></span>
</div>
<div id="container">
    <canvas id="imageCanvas"></canvas>
    <canvas id="maskCanvas"></canvas>
</div>

<script>
const imageCanvas = document.getElementById('imageCanvas');
const maskCanvas = document.getElementById('maskCanvas');
const imageCtx = imageCanvas.getContext('2d');
const maskCtx = maskCanvas.getContext('2d');

const img = new Image();
img.src = 'data:image/png;base64,{img_str}';

img.onload = function() {{
    const maxWidth = Math.min(img.width, 1200);
    const scale = maxWidth / img.width;
    const width = img.width * scale;
    const height = img.height * scale;

    imageCanvas.width = width;
    imageCanvas.height = height;
    maskCanvas.width = width;
    maskCanvas.height = height;

    imageCtx.drawImage(img, 0, 0, width, height);

    // Устанавливаем белый фон для маски
    maskCtx.fillStyle = 'white';
    maskCtx.fillRect(0, 0, width, height);
}};

let isDrawing = false;
let brushSize = {brush_size};
let brushOpacity = {brush_opacity};

maskCanvas.addEventListener('mousedown', startDrawing);
maskCanvas.addEventListener('mousemove', draw);
maskCanvas.addEventListener('mouseup', stopDrawing);
maskCanvas.addEventListener('mouseout', stopDrawing);

// Touch events для мобильных устройств
maskCanvas.addEventListener('touchstart', handleTouch);
maskCanvas.addEventListener('touchmove', handleTouch);
maskCanvas.addEventListener('touchend', stopDrawing);

function startDrawing(e) {{
    isDrawing = true;
    draw(e);
}}

function stopDrawing() {{
    isDrawing = false;
}}

function draw(e) {{
    if (!isDrawing) return;

    const rect = maskCanvas.getBoundingClientRect();
    const x = e.clientX - rect.left;
    const y = e.clientY - rect.top;

    maskCtx.fillStyle = `rgba(0, 0, 0, ${{brushOpacity}})`;
    maskCtx.beginPath();
    maskCtx.arc(x, y, brushSize / 2, 0, Math.PI * 2);
    maskCtx.fill();
}}

function handleTouch(e) {{
    e.preventDefault();
    const touch = e.touches[0];
    const mouseEvent = new MouseEvent(e.type === 'touchstart' ? 'mousedown' : 'mousemove', {{
        clientX: touch.clientX,
        clientY: touch.clientY
    }});
    maskCanvas.dispatchEvent(mouseEvent);
}}

function clearMask() {{
    maskCtx.fillStyle = 'white';
    maskCtx.fillRect(0, 0, maskCanvas.width, maskCanvas.height);
    document.getElementById('status').textContent = '✓ Mask cleared';
    setTimeout(() => {{ document.getElementById('status').textContent = ''; }}, 2000);
}}

function toggleMaskVisibility() {{
    if (maskCanvas.style.opacity === '0') {{
        maskCanvas.style.opacity = '0.5';
    }} else {{
        maskCanvas.style.opacity = '0';
    }}
}}

function saveMask() {{
    // Создаем временный canvas для масштабирования маски обратно к исходному размеру
    const tempCanvas = document.createElement('canvas');
    tempCanvas.width = img.width;
    tempCanvas.height = img.height;
    const tempCtx = tempCanvas.getContext('2d');

    tempCtx.drawImage(maskCanvas, 0, 0, img.width, img.height);

    const maskData = tempCanvas.toDataURL('image/png');

    google.colab.kernel.invokeFunction('notebook.save_mask', [maskData], {{}});

    document.getElementById('status').textContent = '✓ Mask saved!';
    setTimeout(() => {{ document.getElementById('status').textContent = ''; }}, 2000);
}}
</script>
</body>
</html>
"""

def save_mask(mask_data):
    mask_data = mask_data.split(',')[1]
    mask_bytes = base64.b64decode(mask_data)
    mask_image = Image.open(BytesIO(mask_bytes))

    mask_image.save('/content/inpaint_mask.png')
    print("✅ Mask saved to /content/inpaint_mask.png")

output.register_callback('notebook.save_mask', save_mask)

display(HTML(html_code))

print("🎨 Draw on the image to create inpainting mask")
print("   Black areas = will be inpainted")
print("   White areas = will be kept unchanged")

In [ ]:
#@title ##**Run** { display-mode: "form" }

import os
import sys
import time
import torch
import warnings
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, DDIMScheduler, DPMSolverMultistepScheduler
from diffusers.models import AutoencoderKL
from PIL import Image, ImageFilter, ImageOps
import cv2
import numpy as np
import random
import math
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

os.chdir('/content/Tile-Upscaler')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_scheduler(scheduler_name, config):
    if scheduler_name == "DDIM":
        return DDIMScheduler.from_config(config)
    elif scheduler_name == "DPM++ 3M SDE Karras":
        return DPMSolverMultistepScheduler.from_config(config, algorithm_type="sde-dpmsolver++", use_karras_sigmas=True)
    elif scheduler_name == "DPM++ 3M Karras":
        return DPMSolverMultistepScheduler.from_config(config, algorithm_type="dpmsolver++", use_karras_sigmas=True)
    else:
        raise ValueError(f"Unknown scheduler: {scheduler_name}")

if 'inpaint_pipe' not in globals():
    print("🔄 Loading inpainting models...")

    from diffusers.utils import logging as diffusers_logging
    diffusers_logging.set_verbosity_error()

    controlnet = ControlNetModel.from_single_file(
        "models/ControlNet/control_v11f1e_sd15_tile.pth",
        torch_dtype=torch.float16
    )

    inpaint_pipe = StableDiffusionControlNetInpaintPipeline.from_single_file(
        "models/models/Stable-diffusion/juggernaut_reborn.safetensors",
        controlnet=controlnet,
        torch_dtype=torch.float16,
        use_safetensors=True,
        safety_checker=None
    )

    vae = AutoencoderKL.from_single_file(
        "models/VAE/vae-ft-mse-840000-ema-pruned.safetensors",
        torch_dtype=torch.float16
    )
    inpaint_pipe.vae = vae

    inpaint_pipe.load_textual_inversion("models/embeddings/verybadimagenegative_v1.3.pt")
    inpaint_pipe.load_textual_inversion("models/embeddings/JuggernautNegative-neg.pt")
    inpaint_pipe.load_lora_weights("models/Lora/SDXLrender_v2.0.safetensors")
    inpaint_pipe.fuse_lora(lora_scale=0.5)
    inpaint_pipe.load_lora_weights("models/Lora/more_details.safetensors")
    inpaint_pipe.fuse_lora(lora_scale=1.)

    inpaint_pipe.enable_freeu(s1=0.9, s2=0.2, b1=1.3, b2=1.4)
    inpaint_pipe.to(device)
    inpaint_pipe.set_progress_bar_config(disable=True)

    print("✅ Inpainting models loaded!")

inpaint_pipe.scheduler = get_scheduler(scheduler, inpaint_pipe.scheduler.config)

if not os.path.exists('/content/inpaint_mask.png'):
    raise FileNotFoundError("❌ Mask not found! Please create a mask first using cell 5.")

input_image = Image.open('/content/input_image.png').convert("RGB")
mask_image = Image.open('/content/inpaint_mask.png').convert("L")


mask_image = ImageOps.invert(mask_image)

if mask_blur > 0:
    mask_image = mask_image.filter(ImageFilter.GaussianBlur(radius=mask_blur))

print("🎨 Processing inpainting...")
start_time = time.time()

torch.cuda.empty_cache()

W, H = input_image.size

final_prompt = inpaint_prompt if inpaint_prompt else (custom_prompt if custom_prompt else "masterpiece, best quality, highres, detailed")
final_negative = inpaint_negative_prompt if inpaint_negative_prompt else (custom_negative_prompt if custom_negative_prompt else "low quality, normal quality, ugly, blurry, blur, lowres, bad anatomy, bad hands, cropped, worst quality, verybadimagenegative_v1.3, JuggernautNegative-neg")

def process_inpainting_tile(image, mask, control_image, tile_bbox):
    left, top, right, bottom = tile_bbox

    tile_image = image.crop(tile_bbox)
    tile_mask = mask.crop(tile_bbox)
    tile_control = control_image.crop(tile_bbox)

    options = {
        "prompt": final_prompt,
        "negative_prompt": final_negative,
        "image": tile_image,
        "mask_image": tile_mask,
        "control_image": tile_control,
        "num_inference_steps": num_inference_steps,
        "strength": inpaint_strength,
        "guidance_scale": guidance_scale,
        "controlnet_conditioning_scale": float(inpaint_controlnet_strength),
        "generator": torch.Generator(device=device).manual_seed(random.randint(0, 2147483647)),
    }

    return np.array(inpaint_pipe(**options).images[0])

def create_weight_mask(size, edge_fade=32):
    """Создает маску весов для плавного смешивания тайлов"""
    w, h = size
    weight = np.ones((h, w), dtype=np.float32)

    for i in range(edge_fade):
        alpha = i / edge_fade
        weight[i, :] *= alpha
        weight[h-1-i, :] *= alpha
        weight[:, i] *= alpha
        weight[:, w-1-i] *= alpha

    return weight

if use_tiles and (W > tile_size or H > tile_size):
    num_tiles_x = math.ceil(W / (tile_size - tile_overlap))
    num_tiles_y = math.ceil(H / (tile_size - tile_overlap))

    result = np.zeros((H, W, 3), dtype=np.float32)
    weight_sum = np.zeros((H, W, 1), dtype=np.float32)

    total_tiles = num_tiles_x * num_tiles_y
    tiles_processed = 0

    with tqdm(total=total_tiles, desc="Processing tiles", unit="tile") as pbar:
        for i in range(num_tiles_y):
            for j in range(num_tiles_x):
                left = j * (tile_size - tile_overlap)
                top = i * (tile_size - tile_overlap)
                right = min(left + tile_size, W)
                bottom = min(top + tile_size, H)

                tile_mask_array = np.array(mask_image.crop((left, top, right, bottom)))
                if tile_mask_array.max() == 0:
                    original_tile = np.array(input_image.crop((left, top, right, bottom)))
                    tile_h, tile_w = original_tile.shape[:2]
                    tile_weight = np.ones((tile_h, tile_w), dtype=np.float32)

                    result[top:bottom, left:right] += original_tile * tile_weight[:, :, np.newaxis]
                    weight_sum[top:bottom, left:right] += tile_weight[:, :, np.newaxis]
                    pbar.update(1)
                    continue

                tile_bbox = (left, top, right, bottom)
                result_tile = process_inpainting_tile(input_image, mask_image, input_image, tile_bbox)
                tiles_processed += 1

                tile_h, tile_w = result_tile.shape[:2]
                tile_weight = create_weight_mask((tile_w, tile_h), edge_fade=tile_overlap//4)

                result[top:bottom, left:right] += result_tile * tile_weight[:, :, np.newaxis]
                weight_sum[top:bottom, left:right] += tile_weight[:, :, np.newaxis]

                pbar.update(1)

    result = np.where(weight_sum > 0, result / weight_sum, 0).astype(np.uint8)
    output_image = Image.fromarray(result)

else:
    print("Processing full image...")

    options = {
        "prompt": final_prompt,
        "negative_prompt": final_negative,
        "image": input_image,
        "mask_image": mask_image,
        "control_image": input_image,
        "num_inference_steps": num_inference_steps,
        "strength": inpaint_strength,
        "guidance_scale": guidance_scale,
        "controlnet_conditioning_scale": float(inpaint_controlnet_strength),
        "generator": torch.Generator(device=device).manual_seed(random.randint(0, 2147483647)),
    }

    output_image = inpaint_pipe(**options).images[0]

if face_enhance:
    print("👤 Enhancing faces...")
    if 'lazy_gfpgan' not in globals():
        from basicsr.archs.rrdbnet_arch import RRDBNet

        class LazyGFPGAN:
            def __init__(self, device):
                self.device = device
                self.model = None

            def load_model(self):
                if self.model is None:
                    from gfpgan import GFPGANer
                    model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'
                    self.model = GFPGANer(
                        model_path=model_path,
                        upscale=1,
                        arch='clean',
                        channel_multiplier=2,
                        bg_upsampler=None,
                        device=self.device
                    )

            def enhance(self, img):
                self.load_model()
                img_np = np.array(img)
                img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
                _, _, output = self.model.enhance(img_bgr, has_aligned=False, only_center_face=False, paste_back=True)
                output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
                return Image.fromarray(output_rgb)

        lazy_gfpgan = LazyGFPGAN(device)

    output_image = lazy_gfpgan.enhance(output_image)

output_image.save('/content/inpaint_output.png')

end_time = time.time()

print(f"✅ Done in {end_time - start_time:.1f}s | Size: {output_image.size}")
print(f"💾 Saved to: /content/inpaint_output.png")

In [ ]:
#@title ##**Visualize** { display-mode: "form" }

import os
import PIL.Image
import numpy as np
from IPython.display import HTML, display
import base64
from io import BytesIO

#@markdown ### Visualization Method
visualization_method = "Slider" #@param ["Side-by-Side", "Slider"]

def resize_image_maintain_aspect(image, max_width, target_height=None):
    width, height = image.size
    if width > max_width:
        new_height = int(height * max_width / width)
        image = image.resize((max_width, new_height), PIL.Image.LANCZOS)
    if target_height is not None and image.size[1] != target_height:
        new_width = int(image.size[0] * target_height / image.size[1])
        image = image.resize((new_width, target_height), PIL.Image.LANCZOS)
    return image

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Загружаем изображения
if not os.path.exists('/content/input_image.png') or not os.path.exists('/content/inpaint_output.png'):
    print("❌ Error: Images not found. Please run the processing step first.")
else:
    image_original = PIL.Image.open('/content/input_image.png')
    image_restore = PIL.Image.open('/content/inpaint_output.png')

    if visualization_method == "Side-by-Side":
        max_width = 500
        image_original = resize_image_maintain_aspect(image_original, max_width)
        image_restore = resize_image_maintain_aspect(image_restore, max_width)
        target_height = min(image_original.size[1], image_restore.size[1])
        image_original = resize_image_maintain_aspect(image_original, max_width, target_height)
        image_restore = resize_image_maintain_aspect(image_restore, max_width, target_height)

        combined_width = image_original.size[0] + image_restore.size[0]
        combined_image = PIL.Image.new('RGB', (combined_width, target_height))
        combined_image.paste(image_original, (0, 0))
        combined_image.paste(image_restore, (image_original.size[0], 0))
        display(combined_image)

    else:  # Slider
        max_width = min(image_restore.size[0], 1000)
        image_restore = resize_image_maintain_aspect(image_restore, max_width)
        target_height = image_restore.size[1]
        image_original = resize_image_maintain_aspect(image_original, max_width, target_height)

        if image_original.mode != 'RGB':
            image_original = image_original.convert('RGB')
        if image_restore.mode != 'RGB':
            image_restore = image_restore.convert('RGB')

        original_base64 = image_to_base64(image_original)
        restore_base64 = image_to_base64(image_restore)

        html_code = f"""
        <div style="position: relative; width: {image_restore.size[0]}px; height: {image_restore.size[1]}px; margin: 20px auto;">
            <div style="position: relative; width: 100%; height: 100%; overflow: hidden;">
                <img src="data:image/png;base64,{original_base64}" style="position: absolute; width: 100%; height: 100%;">
                <div style="position: absolute; width: 100%; height: 100%; overflow: hidden; clip-path: inset(0 0 0 50%);">
                    <img src="data:image/png;base64,{restore_base64}" style="position: absolute; width: 100%; height: 100%;">
                </div>
            </div>
            <div class="slider" style="position: absolute; top: 0; bottom: 0; width: 4px; background: white; cursor: ew-resize; left: 50%; box-shadow: 0 0 5px rgba(0,0,0,0.5);">
                <div style="position: absolute; top: 50%; transform: translateY(-50%); width: 20px; height: 20px; background: white; border-radius: 50%; left: -8px;"></div>
            </div>
        </div>
        <script>
            (function() {{
                const slider = document.querySelector('.slider');
                if (!slider) return;

                let isDragging = false;
                const container = slider.parentElement.querySelector('div:nth-child(1)');
                const clipDiv = container.querySelector('div');

                slider.addEventListener('mousedown', (e) => {{
                    isDragging = true;
                    e.preventDefault();
                }});

                document.addEventListener('mouseup', () => {{
                    isDragging = false;
                }});

                document.addEventListener('mousemove', (e) => {{
                    if (!isDragging) return;
                    const rect = container.getBoundingClientRect();
                    let x = e.clientX - rect.left;
                    if (x < 0) x = 0;
                    if (x > rect.width) x = rect.width;
                    const percentage = (x / rect.width) * 100;
                    slider.style.left = percentage + '%';
                    clipDiv.style.clipPath = `inset(0 0 0 ${{percentage}}%)`;
                }});

                slider.addEventListener('touchstart', (e) => {{
                    isDragging = true;
                    e.preventDefault();
                }});

                document.addEventListener('touchend', () => {{
                    isDragging = false;
                }});

                document.addEventListener('touchmove', (e) => {{
                    if (!isDragging) return;
                    const rect = container.getBoundingClientRect();
                    let x = e.touches[0].clientX - rect.left;
                    if (x < 0) x = 0;
                    if (x > rect.width) x = rect.width;
                    const percentage = (x / rect.width) * 100;
                    slider.style.left = percentage + '%';
                    clipDiv.style.clipPath = `inset(0 0 0 ${{percentage}}%)`;
                }});
            }})();
        </script>
        """

        display(HTML(html_code))

In [ ]:
#@title ##**Download** { display-mode: "form" }

from google.colab import files
import os

#@markdown ### Download Options
download_result = True  #@param {type:"boolean"}
download_mask = False  #@param {type:"boolean"}

if download_result and os.path.exists('/content/inpaint_output.png'):
    print("💾 Downloading inpainted image...")
    files.download('/content/inpaint_output.png')
    print("✅ Inpainted image download started!")
else:
    if not os.path.exists('/content/inpaint_output.png'):
        print("❌ No inpainted image found! Please run the inpainting step first.")

if download_mask and os.path.exists('/content/inpaint_mask.png'):
    print("💾 Downloading mask...")
    files.download('/content/inpaint_mask.png')
    print("✅ Mask download started!")
else:
    if download_mask and not os.path.exists('/content/inpaint_mask.png'):
        print("❌ No mask found!")